In [2]:
# Initial Imports
import pandas as pd
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from pathlib import Path
import hvplot.pandas
from MCForecastTools import MCSimulation


# Import S&P 500 stock ticker list
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_stocks = table[0]
df_stocks = df_stocks.drop(columns=["Security", "GICS Sub-Industry", "Headquarters Location", "Date added", "CIK", "Founded"])

df_stocks.head()
# df_stocks.to_csv('S&P500-Info.csv')
# df_stocks.to_csv('S&P500-Symbols.csv', columns=['Symbol']['GICS Sector'])
# df_tickers = df_stocks['Symbol']['GICS Sector']
# df_tickers.head()


,Symbol,GICS Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


In [3]:
load_dotenv()

True

In [4]:
df_stocks = df_stocks.rename(columns={
    "Symbol": "symbol"
})
df_stocks.head()

,symbol,GICS Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology


In [5]:
# Import historical price performance for S&P 500 stocks over the last 7 years
# timeframe = '1Day'
# start_date = pd.Timestamp('2016-06-01', tz='America/New_York').isoformat()
# end_date = pd.Timestamp('2023-04-30', tz='America/New_York').isoformat()

# alpaca_api_key = os.getenv("ALPACA_API_KEY")
# alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# alpaca = tradeapi.REST(
#    alpaca_api_key,
#     alpaca_secret_key,
#     api_version="v2"
# )

# tickers = df_stocks['symbol'][:500].tolist()

# # tickers = df_stocks['Symbol'][101:200].tolist()

# df_sp500_hist = alpaca.get_bars(
#     tickers,
#     timeframe,
#     start = start_date,
#     end = end_date
# ).df

In [6]:
# df_sp500_hist

In [7]:
# Placing Data into CSV file
# df_sp500_hist.to_csv('sp500_hist.csv', index=True)


In [8]:
df_sp500_hist = pd.read_csv('sp500_hist.csv')
df_sp500_hist

,timestamp,open,high,low,close,volume,trade_count,vwap,symbol
0,2016-06-01 04:00:00+00:00,31.94,32.260,31.7300,31.95,6416243,42250,31.952436,AAL
1,2016-06-02 04:00:00+00:00,31.67,32.070,31.5799,31.67,6813295,40265,31.803351,AAL
2,2016-06-03 04:00:00+00:00,31.59,31.600,30.3700,30.81,12424133,67650,30.729925,AAL
3,2016-06-06 04:00:00+00:00,30.77,31.300,30.3800,31.26,8335439,53642,30.976306,AAL
4,2016-06-07 04:00:00+00:00,31.46,32.870,31.1000,32.36,12424206,72297,32.405641,AAL
...,...,...,...,...,...,...,...,...,...
854605,2023-04-24 04:00:00+00:00,288.34,290.335,286.3800,288.19,244305,8898,288.013010,ZBRA
854606,2023-04-25 04:00:00+00:00,290.34,293.480,283.9650,284.10,297760,10150,287.351108,ZBRA
854607,2023-04-26 04:00:00+00:00,284.90,286.280,279.5900,280.42,214911,9237,282.230909,ZBRA
854608,2023-04-27 04:00:00+00:00,283.51,285.040,278.2100,284.12,234309,8046,282.403446,ZBRA


In [9]:
# Date column Normalisation
df_sp500_hist = df_sp500_hist.reset_index()

df_sp500_hist = df_sp500_hist.rename(columns={
    "timestamp": "date"
})

#  index_col="Date", infer_datetime_format=True, parse_dates=True)

In [10]:
# Delete Unnecessary Columns

df_sp500_hist = df_sp500_hist.drop(columns=["open", "high", "low", "volume", "trade_count", "vwap", "index"])
df_sp500_hist

,date,close,symbol
0,2016-06-01 04:00:00+00:00,31.95,AAL
1,2016-06-02 04:00:00+00:00,31.67,AAL
2,2016-06-03 04:00:00+00:00,30.81,AAL
3,2016-06-06 04:00:00+00:00,31.26,AAL
4,2016-06-07 04:00:00+00:00,32.36,AAL
...,...,...,...
854605,2023-04-24 04:00:00+00:00,288.19,ZBRA
854606,2023-04-25 04:00:00+00:00,284.10,ZBRA
854607,2023-04-26 04:00:00+00:00,280.42,ZBRA
854608,2023-04-27 04:00:00+00:00,284.12,ZBRA


In [11]:
df_sp500_hist.set_index('date', inplace=True)


In [12]:
df_sp500_hist.head()

,close,symbol
date,,
2016-06-01 04:00:00+00:00,31.95,AAL
2016-06-02 04:00:00+00:00,31.67,AAL
2016-06-03 04:00:00+00:00,30.81,AAL
2016-06-06 04:00:00+00:00,31.26,AAL
2016-06-07 04:00:00+00:00,32.36,AAL


In [13]:
df_sp500_hist['Daily Return'] = df_sp500_hist.groupby('symbol')['close'].pct_change()

In [14]:
display(df_sp500_hist.head())
display(df_sp500_hist.tail())

,close,symbol,Daily Return
date,,,
2016-06-01 04:00:00+00:00,31.95,AAL,NaN
2016-06-02 04:00:00+00:00,31.67,AAL,-0.008764
2016-06-03 04:00:00+00:00,30.81,AAL,-0.027155
2016-06-06 04:00:00+00:00,31.26,AAL,0.014606
2016-06-07 04:00:00+00:00,32.36,AAL,0.035189


,close,symbol,Daily Return
date,,,
2023-04-24 04:00:00+00:00,288.19,ZBRA,0.001007
2023-04-25 04:00:00+00:00,284.10,ZBRA,-0.014192
2023-04-26 04:00:00+00:00,280.42,ZBRA,-0.012953
2023-04-27 04:00:00+00:00,284.12,ZBRA,0.013194
2023-04-28 04:00:00+00:00,288.03,ZBRA,0.013762


In [15]:


# Calculate Daily Returns
# df_sp500_returns = df_sp500.pct_change()
# df_sp500_returns.head(100)


# Drop nulls

df_sp500_hist = df_sp500_hist.dropna().copy()
df_sp500_hist

# Rename `Close` Column to be specific to this portfolio.
# df_sp500_returns = df_sp500_returns.rename(columns={
#     "close": "daily_returns"
# })

df_sp500_hist.head()



,close,symbol,Daily Return
date,,,
2016-06-02 04:00:00+00:00,31.67,AAL,-0.008764
2016-06-03 04:00:00+00:00,30.81,AAL,-0.027155
2016-06-06 04:00:00+00:00,31.26,AAL,0.014606
2016-06-07 04:00:00+00:00,32.36,AAL,0.035189
2016-06-08 04:00:00+00:00,32.51,AAL,0.004635


In [16]:
# Annualized Sharpe Ratios

## 1 sharpe_ratios = ((combined_df.mean()-combined_df['rf_rate'].mean()) * 252) / (combined_df.std() * np.sqrt(252))
##  sharpe_ratios

## 2  sharpe_ratios = (all_returns.mean() * 252) / (all_portfolio_std * np.sqrt(252))
## 2 sharpe_ratios.head()

# joined_std = joined_portfolios.std()

sharpe_ratios = (df_sp500_hist.groupby('symbol')['Daily Return'].mean() * 252) / (df_sp500_hist.groupby('symbol')['Daily Return'].std() * np.sqrt(252))
df_sharpe_ratios = pd.DataFrame(sharpe_ratios)
df_sharpe_ratios = df_sharpe_ratios.rename(columns= {'Daily Return': 'Sharpe Ratio'})
df_stocks = df_stocks.set_index(['symbol'])
display(df_sharpe_ratios.head())
display(df_stocks)

,Sharpe Ratio
symbol,
A,0.701903
AAL,0.050561
AAP,0.090475
AAPL,0.513076
ABBV,0.609853


,GICS Sector
symbol,
MMM,Industrials
AOS,Industrials
ABT,Health Care
ABBV,Health Care
ACN,Information Technology
...,...
YUM,Consumer Discretionary
ZBRA,Information Technology
ZBH,Health Care


In [17]:

df_sharpe_ratios = pd.concat([df_sharpe_ratios, df_stocks], axis = "columns", join = "inner")
df_sharpe_ratios.head()
# Combined_Returns = pd.concat([sp500_df, ar_df, wr_df], axis = "columns", join = "inner")

,Sharpe Ratio,GICS Sector
symbol,,
A,0.701903,Health Care
AAL,0.050561,Industrials
AAP,0.090475,Consumer Discretionary
AAPL,0.513076,Information Technology
ABBV,0.609853,Health Care


In [18]:
# Visualize the sharpe ratios as a bar plot
df_sharpe_ratios.hvplot.bar()

:Bars   [symbol]   (Sharpe Ratio)

In [19]:
df_sharpe_ratios= df_sharpe_ratios.reset_index()

In [20]:
df_top_sharpe_ratios = pd.DataFrame(df_sharpe_ratios.groupby(['GICS Sector', 'symbol'])['Sharpe Ratio'].max().sort_values(ascending=False))
df_top_sharpe_ratios = df_top_sharpe_ratios.head(10)
df_top_sharpe_ratios

Sharpe Ratio
GICS Sector            symbol              
Health Care            GEHC        2.412991
Information Technology ENPH        1.178873
                       CDNS        1.107632
                       SNPS        1.088469
                       AMD         1.045843
Industrials            PWR         1.036131
Financials             AJG         1.034337
Industrials            CARR        1.019958
Health Care            LLY         1.014629
                       PODD        0.997169

In [21]:
max_sharpe_indices = df_sharpe_ratios.groupby('GICS Sector')['Sharpe Ratio'].idxmax()

In [22]:
df_sector = df_sharpe_ratios.loc[max_sharpe_indices]
df_sector

,symbol,Sharpe Ratio,GICS Sector
442,TMUS,0.780146,Communication Services
171,ETSY,0.873457,Consumer Discretionary
111,COST,0.871412,Consumer Staples
313,MPC,0.621992,Energy
20,AJG,1.034337,Financials
200,GEHC,2.412991,Health Care
385,PWR,1.036131,Industrials
161,ENPH,1.178873,Information Technology
124,CTVA,0.818468,Materials
406,SBAC,0.651161,Real Estate


In [23]:
# Get 5 years' worth of historical data for SPY and AGG
df_stock_hist = pd.read_csv('sp500_hist.csv')
df_stock_hist = df_stock_hist.set_index(['timestamp'])
# Reorganize the DataFrame
# Separate ticker data
TMUS = df_stock_hist[df_stock_hist['symbol'] == 'TMUS'].drop('symbol', axis = 1) 
ETSY = df_stock_hist[df_stock_hist['symbol'] == 'ETSY'].drop('symbol', axis = 1)
COST = df_stock_hist[df_stock_hist['symbol'] == 'COST'].drop('symbol', axis = 1)
MPC = df_stock_hist[df_stock_hist['symbol'] == 'MPC'].drop('symbol', axis = 1)
AJG = df_stock_hist[df_stock_hist['symbol'] == 'AJG'].drop('symbol', axis = 1)
GECH = df_stock_hist[df_stock_hist['symbol'] == 'GECH'].drop('symbol', axis = 1)
PWR = df_stock_hist[df_stock_hist['symbol'] == 'PWR'].drop('symbol', axis = 1)
ENPH = df_stock_hist[df_stock_hist['symbol'] == 'ENPH'].drop('symbol', axis = 1)
CTVA = df_stock_hist[df_stock_hist['symbol'] == 'CTVA'].drop('symbol', axis = 1)
CEG = df_stock_hist[df_stock_hist['symbol'] == 'CEG'].drop('symbol', axis = 1)
SBAC = df_stock_hist[df_stock_hist['symbol'] == 'SBAC'].drop('symbol', axis = 1)

# Concatenate the ticker DataFrames
df_stock_data = pd.concat([TMUS,ETSY, COST, MPC, AJG, GECH, PWR, ENPH, CTVA, CEG, SBAC]
                          , axis = 1, keys = ('TMUS','ETSY', 'COST', 'MPC', 'AJG', 'GECH', 'PWR', 'ENPH', 'CTVA', 'CEG', 'SBAC'))

# Display sample data
display(df_stock_data.head())
display(df_stock_data.tail())

TMUS                                              \
                            open    high     low  close   volume trade_count   
timestamp                                                                      
2016-06-01 04:00:00+00:00  42.50  42.880  42.420  42.62  4443732       24278   
2016-06-02 04:00:00+00:00  42.75  43.230  42.430  43.19  4664487       26003   
2016-06-03 04:00:00+00:00  43.18  43.825  42.930  43.63  3492011       23838   
2016-06-06 04:00:00+00:00  43.56  43.820  42.975  43.07  5356814       29238   
2016-06-07 04:00:00+00:00  43.25  43.975  43.170  43.74  2628548       19182   

                                      ETSY              ...    CEG  \
                                vwap  open  high   low  ... volume   
timestamp                                               ...          
2016-06-01 04:00:00+00:00  42.627605  9.21  9.69  9.10  ...    NaN   
2016-06-02 04:00:00+00:00  43.039864  9.36  9.41  8.98  ...    NaN   
2016-06-03 04:00:00+00:00  43.394187  9.24  9.31  9.09  ...    NaN   
2016-06-06 04:00:00+00:00  43.292048  9.23  9.66  9.19  ...    NaN   
2016-06-07 04:00:00+00:00  43.767777  9.63  9.79  9.50  ...    NaN   

                                              SBAC                           \
                          trade_count vwap    open    high      low   close   
timestamp                                                                     
2016-06-01 04:00:00+00:00         NaN  NaN   99.81  100.47   98.250   98.36   
2016-06-02 04:00:00+00:00         NaN  NaN   97.94  101.83   97.940  101.75   
2016-06-03 04:00:00+00:00         NaN  NaN  101.68  102.12  100.170  102.00   
2016-06-06 04:00:00+00:00         NaN  NaN  102.47  102.62  101.950  102.15   
2016-06-07 04:00:00+00:00         NaN  NaN  102.72  104.64  102.515  104.42   

                                                            
                            volume trade_count        vwap  
timestamp                                                   
2016-06-01 04:00:00+00:00  1195254       13209   99.085638  
2016-06-02 04:00:00+00:00  1107879        9797  101.100791  
2016-06-03 04:00:00+00:00   900134        7168  101.791644  
2016-06-06 04:00:00+00:00   601692        6324  102.288878  
2016-06-07 04:00:00+00:00   861091        9319  104.070868  

[5 rows x 77 columns]

TMUS                                       \
                             open      high       low   close   volume   
timestamp                                                                
2023-04-24 04:00:00+00:00  146.13  146.7699  145.6200  145.86  3889329   
2023-04-25 04:00:00+00:00  146.27  149.1800  146.1950  148.00  6396358   
2023-04-26 04:00:00+00:00  146.19  148.9400  145.6625  148.28  4523095   
2023-04-27 04:00:00+00:00  148.85  150.4500  148.7600  149.94  6332780   
2023-04-28 04:00:00+00:00  146.50  147.0200  143.3450  143.90  9337770   

                                                      ETSY                   \
                          trade_count        vwap     open    high      low   
timestamp                                                                     
2023-04-24 04:00:00+00:00       42503  146.096546  102.630  103.25  101.565   
2023-04-25 04:00:00+00:00       57061  148.092733  101.730  102.32   98.110   
2023-04-26 04:00:00+00:00       53139  148.345059   99.350  100.64   97.350   
2023-04-27 04:00:00+00:00       75846  149.882032  100.215  101.38   99.240   
2023-04-28 04:00:00+00:00      116142  144.496454  100.690  101.20   98.710   

                           ...        CEG                           SBAC  \
                           ...     volume trade_count       vwap    open   
timestamp                  ...                                             
2023-04-24 04:00:00+00:00  ...  1094513.0     15527.0  76.948661  256.59   
2023-04-25 04:00:00+00:00  ...  1722751.0     22275.0  76.729842  254.43   
2023-04-26 04:00:00+00:00  ...  3100641.0     32907.0  76.006568  250.41   
2023-04-27 04:00:00+00:00  ...  1865730.0     20120.0  76.488163  251.36   
2023-04-28 04:00:00+00:00  ...  1847125.0     21850.0  76.979594  255.63   

                                                                        \
                              high     low   close  volume trade_count   
timestamp                                                                
2023-04-24 04:00:00+00:00  259.000  253.65  255.62  404479       12730   
2023-04-25 04:00:00+00:00  255.860  251.08  251.62  586864       14754   
2023-04-26 04:00:00+00:00  252.545  248.95  250.20  613522       16953   
2023-04-27 04:00:00+00:00  257.080  251.10  256.85  501320       12791   
2023-04-28 04:00:00+00:00  261.600  255.01  260.89  716460       18948   

                                       
                                 vwap  
timestamp                              
2023-04-24 04:00:00+00:00  255.558056  
2023-04-25 04:00:00+00:00  252.297596  
2023-04-26 04:00:00+00:00  250.414298  
2023-04-27 04:00:00+00:00  255.700326  
2023-04-28 04:00:00+00:00  260.122909  

[5 rows x 77 columns]

In [24]:
MC_even_dist = MCSimulation(
    portfolio_data = df_stock_data,
    weights = [(1/11),(1/11),(1/11),(1/11),(1/11),(1/11),(1/11),(1/11),(1/11),(1/11),(1/11), ],
    num_simulation = 500,
    num_trading_days = 252*30
)

MC_even_dist.portfolio_data.head()

TMUS                                              \
                            open    high     low  close   volume trade_count   
timestamp                                                                      
2016-06-01 04:00:00+00:00  42.50  42.880  42.420  42.62  4443732       24278   
2016-06-02 04:00:00+00:00  42.75  43.230  42.430  43.19  4664487       26003   
2016-06-03 04:00:00+00:00  43.18  43.825  42.930  43.63  3492011       23838   
2016-06-06 04:00:00+00:00  43.56  43.820  42.975  43.07  5356814       29238   
2016-06-07 04:00:00+00:00  43.25  43.975  43.170  43.74  2628548       19182   

                                                   ETSY        ...  CEG  \
                                vwap daily_return  open  high  ... vwap   
timestamp                                                      ...        
2016-06-01 04:00:00+00:00  42.627605          NaN  9.21  9.69  ...  NaN   
2016-06-02 04:00:00+00:00  43.039864     0.013374  9.36  9.41  ...  NaN   
2016-06-03 04:00:00+00:00  43.394187     0.010188  9.24  9.31  ...  NaN   
2016-06-06 04:00:00+00:00  43.292048    -0.012835  9.23  9.66  ...  NaN   
2016-06-07 04:00:00+00:00  43.767777     0.015556  9.63  9.79  ...  NaN   

                                          SBAC                           \
                          daily_return    open    high      low   close   
timestamp                                                                 
2016-06-01 04:00:00+00:00          NaN   99.81  100.47   98.250   98.36   
2016-06-02 04:00:00+00:00          NaN   97.94  101.83   97.940  101.75   
2016-06-03 04:00:00+00:00          NaN  101.68  102.12  100.170  102.00   
2016-06-06 04:00:00+00:00          NaN  102.47  102.62  101.950  102.15   
2016-06-07 04:00:00+00:00          NaN  102.72  104.64  102.515  104.42   

                                                                         
                            volume trade_count        vwap daily_return  
timestamp                                                                
2016-06-01 04:00:00+00:00  1195254       13209   99.085638          NaN  
2016-06-02 04:00:00+00:00  1107879        9797  101.100791     0.034465  
2016-06-03 04:00:00+00:00   900134        7168  101.791644     0.002457  
2016-06-06 04:00:00+00:00   601692        6324  102.288878     0.001471  
2016-06-07 04:00:00+00:00   861091        9319  104.070868     0.022222  

[5 rows x 88 columns]

In [28]:
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.


KeyboardInterrupt: 